In [6]:
from typing import Dict, Union
from IPython import get_ipython
import autogen

class IPythonUserProxyAgent(autogen.UserProxyAgent):
    def generate_init_message(self, *args, **kwargs) -> Union[str, Dict]:
        return super().generate_init_message(*args, **kwargs) + """If you suggest code, the code will be executed in IPython."""
    def run_code(self, code, **kwargs):
        result = self._ipython.run_cell("%%capture --no-display cap\n" + code)
        log = self._ipython.ev("cap.stdout")
        log += self._ipython.ev("cap.stderr")
        if result.result is not None:
            log += str(result.result)
        exitcode = 0 if result.success else 1
        if result.error_before_exec is not None:
            log += f"\n{result.error_before_exec}"
            exitcode = 1
        if result.error_in_exec is not None:
            log += f"\n{result.error_in_exec}"
            exitcode = 1
        return exitcode, log, None
    

In [7]:
import autogen

from autogen import AssistantAgent, UserProxyAgent,oai
config_list = [
    {
        "model": "codellama-7b-instruct.Q5_K_M.gguf",
        "api_base": "http://127.0.0.1:8000/v1",
        "api_type": "open_ai",
        "api_key": "NULL",
    }
]

llm_config = {"config_list": config_list, "seed": 42, "request_timeout": 2028,}

# Create assistant agent
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    system_message="""Engineer. You follow an approved plan. 
    You write python/shell code to solve tasks. 
    Wrap the code in a code block that specifies the script type. 
    The user can't modify your code. So do not suggest incomplete 
    code which requires others to modify. Don't use a code block 
    if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. Do not ask 
    others to copy and paste the result. Check the execution result 
    returned by the executor. If the result indicates there is 
    an error, fix the error and output the code again. Suggest
    the full code instead of partial code or code changes. If
    the error can't be fixed or if the task is not solved even
    after the code is executed successfully, analyze the 
    problem, revisit your assumption, collect additional info
    you need, and think of a different approach to try.
""",
)

ipy_user = IPythonUserProxyAgent(
    "ipython_user_proxy",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get(
    "content", "").rstrip().endswith(
    "TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
)
# the assistant receives a message from the user, which contains the task description
ipy_user.initiate_chat(
    assistant,
    message="""plot a stacked area chart
     visualizing the yearly high and low 
     prices of Apple (AAPL), Google 
     (GOOGL), Amazon (AMZN), and Microsoft (MSFT)
    use yfinance,matplotlib,pandas packages
    """,
)

ipython_user_proxy (to assistant):

plot a stacked area chart
     visualizing the yearly high and low 
     prices of Apple (AAPL), Google 
     (GOOGL), Amazon (AMZN), and Microsoft (MSFT)
    use yfinance,matplotlib,pandas packages
    If you suggest code, the code will be executed in IPython.

--------------------------------------------------------------------------------
assistant (to ipython_user_proxy):

  Here is an example of how to plot a stacked area chart using Python's `yfinance`, `matplotlib`, and `pandas` packages:
```
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

# Get the historical data for Apple, Google, Amazon, and Microsoft
data = yf.download('AAPL GOOGL AMZN MSFT', start='2010-01-01', end='2023-02-26')

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Drop any rows with missing values
df.dropna(inplace=True)

# Create a new column for the year
df['year'] = df.index.date.map(lambda x: x.year)

# Group the data by yea

In [10]:
from autogen import AssistantAgent, UserProxyAgent,oai, config_list_openai_aoai, ChatCompletion
config_list = [
    {
        "model": "codellama-7b-instruct.Q5_K_M.gguf",
        "api_base": "http://127.0.0.1:8000/v1",
        "api_type": "open_ai",
        "api_key": "NULL",
    }
]

llm_config = {"config_list": config_list, "seed": 42, "request_timeout": 2028,}

planner = AssistantAgent(
    name="planner",
    llm_config={"config_list": config_list},
    # the default system message of the AssistantAgent is overwritten here
    system_message="""You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. 
    Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step that can be implemented by writing code. 
    For example, browsing the web can be implemented by writing code that reads and prints the content of a web page. Finally, inspect the execution result. 
    Shell code should be formatted as `sh`
    If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix."""
)
planner_user = UserProxyAgent(
    name="planner_user",
    llm_config=llm_config,
    max_consecutive_auto_reply=0,  # terminate without auto-reply
    human_input_mode="NEVER",
)

def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    # return the last message received from the planner
    return planner_user.last_message()["content"]


# create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(
    name="user_proxy",
    llm_config=llm_config,
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "planning"},
    function_map={"ask_planner": ask_planner},
)

ChatCompletion.start_logging( compact=False)

# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""Plot a chart of AMZN, AAPL and INTC stock
 price change YTD.use yfinance,matplotlib,pandas packages 
 and save to stock_price_ytd.png.""",
)

user_proxy (to assistant):

Plot a chart of AMZN, AAPL and INTC stock
 price change YTD.use yfinance,matplotlib,pandas packages 
 and save to stock_price_ytd.png.

--------------------------------------------------------------------------------
assistant (to user_proxy):

  ```
# Import necessary libraries
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

# Get the stock prices for AMZN, AAPL, and INTC
data = yf.download('AMZN', 'AAPL', 'INTC')['Close']

# Create a DataFrame with the data
df = pd.DataFrame(data, columns=['AMZN', 'AAPL', 'INTC'])

# Set the index to be the date
df.index = df.index.date

# Resample the data to yearly frequency
df_yearly = df.resample('1Y').sum()

# Plot the stock prices
plt.plot(df_yearly)
plt.xlabel('Year')
plt.ylabel('Stock Price ($)')
plt.title('Stock Prices for AMZN, AAPL, and INTC YTD')
plt.savefig('stock_price_ytd.png')

# Terminate the program
print("TERMINATE")
```
This code will download the stock prices for AMZN, AAPL, 

DockerException: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))

In [21]:
import os
from autogen import AssistantAgent, UserProxyAgent

assistant = AssistantAgent(name="assistant", llm_config=llm_config)
user_proxy = UserProxyAgent(name="user_proxy", llm_config=llm_config)

user_proxy.initiate_chat(
    assistant,
    message="""Hello, today you are my data analyst assistant and you should help me visualize data, make predictions, and explain your thinking.""",
)
response = oai.Completion.create(
    config_list = [
        {
            "model": "codellama-7b-instruct.Q5_K_M.gguf",
            "api_base": "http://127.0.0.1:8000/v1",
            "api_type": "open_ai",
            "api_key": "NULL",
        }
    ],

    llm_config = {"config_list": config_list, "seed": 42, "request_timeout": 2028,},
    prompt="Hi",
)
print(response)


user_proxy (to assistant):

Hello, today you are my data analyst assistant and you should help me visualize data, make predictions, and explain your thinking.

--------------------------------------------------------------------------------
assistant (to user_proxy):

  Hello! As a data analyst, I can assist you with tasks such as data visualization, making predictions based on data, and explaining my reasoning. Here's how we can work together:

1. Provide the data: Please provide me with the data you would like to analyze, including any relevant tables, charts, or graphs.
2. Define the task: Please let me know what specific analysis or prediction tasks you would like me to perform on the provided data.
3. Assess the data: I will assess the data and identify any potential issues or limitations that may impact our analysis.
4. Visualize the data: Based on your instructions, I will create visualizations of the data using tools such as charts, graphs, or maps.
5. Make predictions: Once th

KeyboardInterrupt: 